In [1]:
from functions.db_util import *
from functions.ollama_util import *
from functions.trans_formers_util import *
from functions.query_builder import QueryBuilder

c:\Users\JourneyQ\anaconda3\envs\ai_agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conn_info = {
    "host": "localhost",  # ไม่ใช่ localhost!
    "port": "5432",
    "user": "admin",
    "password": "admin",
    "dbname": "vector_db"
}

db = MyDbConnect(conn_info)

In [4]:
db.connect()

### set up 

docker run -d --name pgvector-db -e POSTGRES_USER=admin -e POSTGRES_PASSWORD=admin -e POSTGRES_DB=vector_db -v pgvector_data:/var/lib/postgresql/data -p 5432:5432 pgvector/pgvector:pg17

In [4]:
# เพิ่ม extension ให้ pg
result, error = db.execute("Create extension vector;")

In [7]:
# เพิ่ม vector table 
result, err = db.execute("""
    create table if not exists documents (
        id serial primary key,
        content text,
        embedding vector(1024)
        )
    """)

In [8]:
db.commit()

### Embed

In [5]:
embeder = TransFormersEmbedding("BAAI/bge-m3")

In [8]:
text = "sale_items เป็นตารางเก็บข้อมูลรายการสินค้าในคำสั่งซื้อ"
vec, err = embeder.embedding_text(text)

In [9]:
mod = [{
    "content": text,
    "embedding": vec.tolist()
}]

In [10]:
sql, val = QueryBuilder.insert_data_query("documents", mod)

In [11]:
db.executemany(sql, val)

(1, None)

In [12]:
db.commit()

In [13]:
db.connect()

### SEARCH

In [14]:
def search_similarity_text(db, embeder, text, r):

    try :
        vec, err = embeder.embedding_text(text)

        query_vec_str = "[" + ", ".join(map(str, vec)) + "]"

        sql_query = QueryBuilder.vector_search()

        res, err = db.execute(sql_query, params=(query_vec_str, r))
        if err :
            raise Exception(err)
        
        return res, None
    except Exception as err :
        return None, err
    

In [16]:
db.connect()

text = "คำสั่งซื้อดูได้ที่ตารางไหน"

res, err = search_similarity_text(db, embeder, text, 5)

db.disconnect()

### RAG

In [ ]:
def RAG_chat(db, embeder , model, text, k=5):

    try:
        res, err = search_similarity_text(db, embeder, text, k)
        if err :
            raise Exception(err)
        
        context = "\n".join( [data[0] for data in res])
        prompt = f"""
            Answer the question based on the following context :\n
            ```\n
            {context}
            ```\n
            Question: {text}
        """
        ans = model.send_user_prompt(prompt)
        
        return ans
    except Exception as err:
        return None, err


In [42]:
chat_model = OllamaChatClient("gemma3:latest")
chat_model.set_system_prompt("คุณคือผู้เชียวชาญด้านการตอบคำถามจากข้อมูลที่มี ถ้าไม่มีคุณจะตอบว่า 'กรรมไม่รู้แหะ'")

In [58]:
text = "คุณรู้อะไรบ้าง"
db.connect()
ans = RAG_chat(db, embeder , chat_model, text, k=5)
print(ans)
db.disconnect()

กรรมไม่รู้แหะ



In [29]:
ans

(None, TypeError("string indices must be integers, not 'str'"))